# Livelike-PMEDMrcpp Bridge: Batched Runs

In [1]:
%load_ext watermark
%watermark

Last updated: 2025-03-20T16:12:28.585340-04:00

Python implementation: CPython
Python version       : 3.12.9
IPython version      : 9.0.2

Compiler    : Clang 18.1.8 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit



In [2]:
import pathlib

import likeness_vitals
import livelike
import numpy
import pandas

import pmedm_legacy

%watermark -w
%watermark -iv

Watermark: 2.5.0

likeness_vitals: 0.1.dev3+g2392110.d20250320
pmedm_legacy   : 0.0.5
livelike       : 1.4.8.dev3+geea70e8
pandas         : 2.2.3
numpy          : 2.2.4



-------------------

## Get census API key

In [3]:
key = likeness_vitals.get_censusapikey(pathlib.Path("..", "..", "livelike", ""))

--------------------
## Create PUMA classes (`livelike.puma`)
### target PUMAs - Knox County, TN

*This step may take several minutes*

In [4]:
pumas = ["4701601", "4701602", "4701603", "4701604"]
mpu = livelike.make_pumas(pumas, cache=True, censusapikey=key)

---------------------------
## Create 4 P-MEDM instances and solve each

In [5]:
pmds = pmedm_legacy.parallel.parallel_solve(
    mpu,
    tr_iter=300,
    tr_tol=1e-4,
)

Using Sparse Hessian Preconditioner
Using Sparse Hessian Preconditioner
Using Sparse Hessian Preconditioner
Using Sparse Hessian Preconditioner
Initializing PMEDM solver
Starting Trust Region Algorithm
 iter          fun     ||grad|| ||try step||      act red     pred red     radius inner iter
    0 -1.41331e-13     0.343387            1  -0.00310982      1.16901          1           0
    1 -1.41331e-13     0.343387         0.25     0.383064     0.386002       0.25           0
    2    -0.383064     0.284839          0.5     0.408097     0.521036        0.5           0
Initializing PMEDM solver
Starting Trust Region Algorithm
 iter          fun     ||grad|| ||try step||      act red     pred red     radius inner iter
    0  1.13465e-13     0.220983            1     -1.55961      1.20199          1           0
    1  1.13465e-13     0.220983         0.25     0.392344     0.394248       0.25           0
    2    -0.392344     0.194507          0.5     0.488141     0.559896        0.5   

-------------------------------
## Diagnostics

In [6]:
ycp_all = []
ych_all = []

for p in mpu:
    print(f"PUMA {p}\n")

    mfrs = pmedm_legacy.moe_fit_rate(
        mpu[p].est_ind,
        mpu[p].est_g2,
        mpu[p].se_g2,
        pmds[p].pmedm_res["almat"],
    )

    print("ACS 90% MOE Fit Rate: " + str(numpy.round(mfrs["moe_fit_rate"], 4)) + "\n")

    Ycomps = mfrs["Ycomp"]
    ycp = Ycomps[Ycomps["constraint"] == "population"]
    ych = Ycomps[Ycomps["constraint"] == "housing_units"]

    acs_totalpop = ycp["acs"].sum()
    pmedm_totalpop = ycp["pmedm"].sum()
    err_totalpop = pmedm_totalpop - acs_totalpop

    acs_totalhu = ych["acs"].sum()
    pmedm_totalhu = ych["pmedm"].sum()
    err_totalhu = pmedm_totalhu - acs_totalhu

    print(f"ACS Total Population: {acs_totalpop}")
    print(f"P-MEDM Total Population: {pmedm_totalpop}")
    print(f"Error : {err_totalpop}")
    print(f"Error (%) : {abs(err_totalpop) / acs_totalpop}\n")

    print(f"ACS Total Housing Units: {acs_totalhu}")
    print(f"P-MEDM Total Units: {pmedm_totalhu}")
    print(f"Error : {pmedm_totalhu - acs_totalhu}")
    print(f"Error (%) : {abs(err_totalhu) / acs_totalhu}\n\n")

    ycp_all += [ycp]
    ych_all += [ych]

PUMA 4701601

ACS 90% MOE Fit Rate: 0.9998

ACS Total Population: 107250
P-MEDM Total Population: 107315.79589745788
Error : 65.79589745787962
Error (%) : 0.0006134815613788309

ACS Total Housing Units: 49264
P-MEDM Total Units: 49290.556244128675
Error : 26.556244128674734
Error (%) : 0.0005390598434693637


PUMA 4701602

ACS 90% MOE Fit Rate: 0.9996

ACS Total Population: 113236
P-MEDM Total Population: 113175.41156171571
Error : -60.58843828429235
Error (%) : 0.0005350633922453315

ACS Total Housing Units: 47525
P-MEDM Total Units: 47636.26087667536
Error : 111.26087667536194
Error (%) : 0.0023411020868040386


PUMA 4701603

ACS 90% MOE Fit Rate: 0.9999

ACS Total Population: 178350
P-MEDM Total Population: 178786.87245946575
Error : 436.8724594657542
Error (%) : 0.002449523181753598

ACS Total Housing Units: 87369
P-MEDM Total Units: 87765.49277844286
Error : 396.49277844285825
Error (%) : 0.004538140283657341


PUMA 4701604

ACS 90% MOE Fit Rate: 0.9999

ACS Total Population: 1578

-------------------------

## Population Synthesis

In [7]:
# TODO: use parallel processing
syps = []
for p in pumas:
    _syp = livelike.homesim.synthesize(
        pmds[p].pmedm_res["almat"],
        mpu[p].est_ind,
        mpu[p].est_g2,
        mpu[p].sporder,
        nsim=1,
        random_state=int(p),
    )
    _syp = _syp.reset_index()
    syps += [_syp]

In [8]:
syps = pandas.concat(syps, axis=0)
syps

,h_id,sim,geoid,count
0,2015000045904,0,470010201001,1
1,2015000050397,0,470010201001,1
2,2015000054551,0,470010201001,1
3,2015000067347,0,470010201001,5
4,2015000089285,0,470010201001,1
...,...,...,...,...
41945,2019HU1396194,0,470930060032,9
41946,2019HU1397336,0,470930060032,1
41947,2019HU1405831,0,470930060032,1
41948,2019HU1406178,0,470930060032,2


---------------------